## LIME

In [ ]:
# Separate 'Applicant ID' from the rest of the DataFrame
applicant_ids = df['Applicant ID']
df_features = df.drop('Applicant ID', axis=1)

# One-hot encode categorical variables
df_encoded = pd.get_dummies(df_features, drop_first=True)

# Scale the numeric features (StandardScaler expects numeric values)
scaler = StandardScaler()
df_encoded_scaled = scaler.fit_transform(df_encoded)

# Add 'Applicant ID' back to the DataFrame
df_encoded_scaled_with_id = pd.DataFrame(df_encoded_scaled, columns=df_encoded.columns)
df_encoded_scaled_with_id['Applicant ID'] = applicant_ids

# Initialize the explainer for classification
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=df_encoded_scaled_with_id.drop('Applicant ID', axis=1).values,
    feature_names=df_encoded.columns,
    class_names=['No Interview', 'Interview'],
    mode='classification',
    discretize_continuous=True
)

# Select an instance to explain
instance_index = 1  # Adjust this to your specific instance
instance = df_encoded_scaled_with_id.drop('Applicant ID', axis=1).iloc[instance_index].values


def predict_res(data_as_np):
    # Reshape the input data to match the original shape
    input_df = pd.DataFrame(data_as_np, columns=df_encoded_scaled_with_id.drop('Applicant ID', axis=1).columns)
    # Add 'Applicant ID' back to the DataFrame
    num_rows = len(input_df)
    input_df['Applicant ID'] = range(1, num_rows + 1)
    # Make predictions using the model_predict function
    predictions_df = model_predict(input_df)
    
    if predictions_df is None:
        print("Error: Unable to make predictions")
        return None
    
    return predictions_df['Interview prediction'].values

# Explain the prediction
exp = explainer.explain_instance(
    instance,
    predict_res, 
    num_features=5, 
    top_labels=1
)
exp.show_in_notebook(show_table=True, show_all=False)